In [1]:
import pandas as pd
from scipy.stats import mannwhitneyu
from itertools import combinations
import numpy as np

In [2]:
ruta="../"
models=["gemma2:2b","gemma2","llama3.2","llama3.1","phi3:medium","phi3"]#,"phi3"]
corpus=["DIAG","SNLI","SICK","Scitail","RTEGLUE"]
grupos=["G1","G2","G3","G4"]

In [3]:
combinationsGs = list(combinations(grupos, 3))
combinationsGs.extend(list(combinations(grupos, 2)))
combinationsGs.extend(grupos)
print(combinationsGs)


[('G1', 'G2', 'G3'), ('G1', 'G2', 'G4'), ('G1', 'G3', 'G4'), ('G2', 'G3', 'G4'), ('G1', 'G2'), ('G1', 'G3'), ('G1', 'G4'), ('G2', 'G3'), ('G2', 'G4'), ('G3', 'G4'), 'G1', 'G2', 'G3', 'G4']


In [4]:
new_data={"model":[],"corpus":[],
          "SKA_DT_avg":[],#"SKA_DT_std":[],
          "Base_avg":[],#"Base_std":[],
          "Comb":[],"Comb_avg":[],"p_value_SKA_DT":[],"p_value_base":[] #"Comb_std":[],
          }

In [5]:
for m in models:
    for c in corpus:
        data_results1=pd.read_pickle(ruta+m+"/"+c+"/answers/cv1/results.pickle")
        data_results2=pd.read_pickle(ruta+m+"/"+c+"/answers/cv2/results.pickle")
        data_results3=pd.read_pickle(ruta+m+"/"+c+"/answers/cv3/results.pickle")
        data_results4=pd.read_pickle(ruta+m+"/"+c+"/answers/cv4/results.pickle")
        data_results5=pd.read_pickle(ruta+m+"/"+c+"/answers/cv5/results.pickle")
        data_results=pd.concat([data_results1,data_results2,data_results3,data_results4,data_results5],axis=0)
        

        ska_dt=data_results[data_results["group"]=="SKA_DT"]["accuracy"]
        ska_dt_avg=data_results[data_results["group"]=="SKA_DT"]["accuracy"].mean()
        ska_dt_std=data_results[data_results["group"]=="SKA_DT"]["accuracy"].std()

        base=data_results[data_results["group"]=="Base"]["accuracy"]
        base_avg=data_results[data_results["group"]=="Base"]["accuracy"].mean()
        base_std=data_results[data_results["group"]=="Base"]["accuracy"].std()

        print(ska_dt)
        for comb in combinationsGs:
            comb_data=data_results[data_results["group"]==comb]["accuracy"]
            comb_avg=data_results[data_results["group"]==comb]["accuracy"].mean()
            comb_std=data_results[data_results["group"]==comb]["accuracy"].std()

            new_data["model"].append(m)
            new_data["corpus"].append(c)
            new_data["Comb"].append(comb)
            new_data["Comb_avg"].append(comb_avg)
            #new_data["Comb_std"].append(comb_std)    
            new_data["SKA_DT_avg"].append(ska_dt_avg)
            #new_data["SKA_DT_std"].append(ska_dt_std)
            new_data["Base_avg"].append(base_avg)
            #new_data["Base_std"].append(base_dt_std)
            new_data["p_value_SKA_DT"].append(mannwhitneyu(comb_data, ska_dt).pvalue)
            new_data["p_value_base"].append(mannwhitneyu(comb_data,base).pvalue)

            

FileNotFoundError: [Errno 2] No such file or directory: '../gemma2:2b/DIAG/answers/cv1/results.pickle'

In [ ]:
temp=pd.DataFrame(new_data)
temp

,model,corpus,SKA_DT_avg,Base_avg,Comb,Comb_avg,p_value_dt,p_value_base
0,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G2, G3)",0.628409,1.192523e-02,2.117666e-02
1,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G2, G4)",0.672727,9.165626e-01,1.160739e-01
2,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G3, G4)",0.673580,1.000000e+00,1.160739e-01
3,gemma2:2b,DIAG,0.673864,0.654830,"(G2, G3, G4)",0.669886,6.004018e-01,9.523810e-02
4,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G2)",0.617045,7.936508e-03,7.936508e-03
...,...,...,...,...,...,...,...,...
415,phi3,RTEGLUE,0.851400,0.849777,"(G3, G4)",0.851600,9.530611e-01,6.451457e-01
416,phi3,RTEGLUE,0.851400,0.849777,G1,0.838500,1.211788e-02,9.459216e-03
417,phi3,RTEGLUE,0.851400,0.849777,G2,0.822900,4.046946e-07,4.456728e-06
418,phi3,RTEGLUE,0.851400,0.849777,G3,0.805800,1.225260e-10,7.997922e-09


In [626]:
valor_sig_sensibility=0.05
valor_sig_robustez=0.01

In [627]:
loss_sig=temp[(~temp["Comb"].isin(grupos)) & (temp["p_value_dt"]<valor_sig_sensibility) & (temp["p_value_base"]<valor_sig_robustez)]

In [628]:
loss_sig["dif"]=(loss_sig["SKA_DT_avg"]-loss_sig["Comb_avg"])*100
loss_sig["dif_base"]=(temp["Comb_avg"]-temp["Base_avg"])*100
loss_sig["missing"]=loss_sig["Comb"].apply(lambda x: set(grupos).difference(set(list(x))))
loss_sig["sensible"]=loss_sig["dif"].apply(lambda x: "sensible" if x>0 else "errorComb" )
loss_sig["robustez"]=loss_sig["dif_base"].apply(lambda x: "robusto" if x>0 else "falla" )
loss_sig

/tmp/ipykernel_2660611/2374206213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loss_sig["dif"]=(loss_sig["SKA_DT_avg"]-loss_sig["Comb_avg"])*100
/tmp/ipykernel_2660611/2374206213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loss_sig["dif_base"]=(temp["Comb_avg"]-temp["Base_avg"])*100
/tmp/ipykernel_2660611/2374206213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,model,corpus,SKA_DT_avg,Base_avg,Comb,Comb_avg,p_value_dt,p_value_base,dif,dif_base,missing,sensible,robustez
4,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G2)",0.617045,7.936508e-03,7.936508e-03,5.681818,-3.778409,"{G3, G4}",sensible,falla
5,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G3)",0.616477,7.936508e-03,7.936508e-03,5.738636,-3.835227,"{G2, G4}",sensible,falla
17,gemma2:2b,SNLI,0.665333,0.630974,"(G2, G3, G4)",0.651367,5.340134e-04,7.195484e-07,1.396667,2.039282,{G1},sensible,robusto
18,gemma2:2b,SNLI,0.665333,0.630974,"(G1, G2)",0.656800,1.808642e-02,9.490809e-09,0.853333,2.582615,"{G3, G4}",sensible,robusto
19,gemma2:2b,SNLI,0.665333,0.630974,"(G1, G3)",0.589367,6.769339e-18,4.919151e-16,7.596667,-4.160718,"{G2, G4}",sensible,falla
...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,phi3,SICK,0.867067,0.820533,"(G1, G2)",0.857800,2.832270e-03,5.268782e-14,0.926667,3.726667,"{G3, G4}",sensible,robusto
383,phi3,SICK,0.867067,0.820533,"(G1, G3)",0.858667,6.087223e-03,3.133510e-14,0.840000,3.813333,"{G2, G4}",sensible,robusto
386,phi3,SICK,0.867067,0.820533,"(G2, G4)",0.853800,1.398416e-03,1.359808e-10,1.326667,3.326667,"{G3, G1}",sensible,robusto
387,phi3,SICK,0.867067,0.820533,"(G3, G4)",0.857133,8.471179e-03,3.810870e-13,0.993333,3.660000,"{G1, G2}",sensible,robusto


In [629]:
valor_sig_error=0.05
valor_sig_ruido=0.01

In [630]:
support=temp[(temp["Comb"].isin(grupos)) & ((temp["p_value_dt"]<valor_sig_error) | (temp["p_value_base"]<valor_sig_ruido))]
support["dif"]=(temp["SKA_DT_avg"]-temp["Comb_avg"])*100
support["dif_base"]=(temp["Base_avg"]-temp["Comb_avg"])*100
support["error"]=support["dif"].apply(lambda x: "errorGs" if x<0 else "ok" )
support["info"]=support["dif_base"].apply(lambda x: "ruido" if x>0 else "información" )
support["support"]=temp["Comb"]
support

/tmp/ipykernel_2660611/4221821194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  support["dif"]=(temp["SKA_DT_avg"]-temp["Comb_avg"])*100
/tmp/ipykernel_2660611/4221821194.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  support["dif_base"]=(temp["Base_avg"]-temp["Comb_avg"])*100
/tmp/ipykernel_2660611/4221821194.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,model,corpus,SKA_DT_avg,Base_avg,Comb,Comb_avg,p_value_dt,p_value_base,dif,dif_base,error,info,support
10,gemma2:2b,DIAG,0.673864,0.654830,G1,0.616152,7.936508e-03,7.936508e-03,5.771127,3.867718,ok,ruido,G1
24,gemma2:2b,SNLI,0.665333,0.630974,G1,0.559849,6.781591e-18,3.883572e-17,10.548414,7.112465,ok,ruido,G1
25,gemma2:2b,SNLI,0.665333,0.630974,G2,0.556333,6.644955e-18,6.428834e-18,10.900000,7.464051,ok,ruido,G2
26,gemma2:2b,SNLI,0.665333,0.630974,G3,0.509715,6.781591e-18,6.561258e-18,15.561792,12.125843,ok,ruido,G3
27,gemma2:2b,SNLI,0.665333,0.630974,G4,0.618181,8.940895e-16,1.352263e-03,4.715275,1.279327,ok,ruido,G4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,phi3,Scitail,0.839800,0.812209,G3,0.844850,7.312853e-02,1.720408e-12,-0.505000,-3.264110,errorGs,información,G3
405,phi3,Scitail,0.839800,0.812209,G4,0.827900,2.875273e-03,4.381783e-05,1.190000,-1.569110,ok,información,G4
416,phi3,RTEGLUE,0.851400,0.849777,G1,0.838500,1.211788e-02,9.459216e-03,1.290000,1.127693,ok,ruido,G1
417,phi3,RTEGLUE,0.851400,0.849777,G2,0.822900,4.046946e-07,4.456728e-06,2.850000,2.687693,ok,ruido,G2


#

In [631]:
data_p=pd.concat([loss_sig,support],axis=0)
data_p

,model,corpus,SKA_DT_avg,Base_avg,Comb,Comb_avg,p_value_dt,p_value_base,dif,dif_base,missing,sensible,robustez,error,info,support
4,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G2)",0.617045,7.936508e-03,7.936508e-03,5.681818,-3.778409,"{G3, G4}",sensible,falla,NaN,NaN,NaN
5,gemma2:2b,DIAG,0.673864,0.654830,"(G1, G3)",0.616477,7.936508e-03,7.936508e-03,5.738636,-3.835227,"{G2, G4}",sensible,falla,NaN,NaN,NaN
17,gemma2:2b,SNLI,0.665333,0.630974,"(G2, G3, G4)",0.651367,5.340134e-04,7.195484e-07,1.396667,2.039282,{G1},sensible,robusto,NaN,NaN,NaN
18,gemma2:2b,SNLI,0.665333,0.630974,"(G1, G2)",0.656800,1.808642e-02,9.490809e-09,0.853333,2.582615,"{G3, G4}",sensible,robusto,NaN,NaN,NaN
19,gemma2:2b,SNLI,0.665333,0.630974,"(G1, G3)",0.589367,6.769339e-18,4.919151e-16,7.596667,-4.160718,"{G2, G4}",sensible,falla,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,phi3,Scitail,0.839800,0.812209,G3,0.844850,7.312853e-02,1.720408e-12,-0.505000,-3.264110,NaN,NaN,NaN,errorGs,información,G3
405,phi3,Scitail,0.839800,0.812209,G4,0.827900,2.875273e-03,4.381783e-05,1.190000,-1.569110,NaN,NaN,NaN,ok,información,G4
416,phi3,RTEGLUE,0.851400,0.849777,G1,0.838500,1.211788e-02,9.459216e-03,1.290000,1.127693,NaN,NaN,NaN,ok,ruido,G1
417,phi3,RTEGLUE,0.851400,0.849777,G2,0.822900,4.046946e-07,4.456728e-06,2.850000,2.687693,NaN,NaN,NaN,ok,ruido,G2


In [632]:
data_p[data_p["corpus"]=="RTEGLUE"]

,model,corpus,SKA_DT_avg,Base_avg,Comb,Comb_avg,p_value_dt,p_value_base,dif,dif_base,missing,sensible,robustez,error,info,support
63,gemma2:2b,RTEGLUE,0.7513,0.745253,"(G2, G3)",0.722800,1.134639e-04,3.435249e-03,2.850000,-2.245276,"{G1, G4}",sensible,falla,NaN,NaN,NaN
196,llama3.2,RTEGLUE,0.7451,0.714400,"(G1, G2, G3)",0.690100,1.634542e-13,1.870093e-03,5.500000,-2.430000,{G4},sensible,falla,NaN,NaN,NaN
200,llama3.2,RTEGLUE,0.7451,0.714400,"(G1, G2)",0.688500,2.873758e-13,2.894026e-03,5.660000,-2.590000,"{G3, G4}",sensible,falla,NaN,NaN,NaN
201,llama3.2,RTEGLUE,0.7451,0.714400,"(G1, G3)",0.647100,5.791931e-18,6.271777e-17,9.800000,-6.730000,"{G2, G4}",sensible,falla,NaN,NaN,NaN
203,llama3.2,RTEGLUE,0.7451,0.714400,"(G2, G3)",0.690100,1.634542e-13,1.870093e-03,5.500000,-2.430000,"{G1, G4}",sensible,falla,NaN,NaN,NaN
271,llama3.1,RTEGLUE,0.7940,0.752900,"(G1, G3)",0.774700,1.052441e-04,1.800896e-05,1.930000,2.180000,"{G2, G4}",sensible,robusto,NaN,NaN,NaN
272,llama3.1,RTEGLUE,0.7940,0.752900,"(G1, G4)",0.773800,2.124448e-04,8.025761e-05,2.020000,2.090000,"{G3, G2}",sensible,robusto,NaN,NaN,NaN
413,phi3,RTEGLUE,0.8514,0.849777,"(G2, G3)",0.827300,2.286246e-05,5.617002e-05,2.410000,-2.247693,"{G1, G4}",sensible,falla,NaN,NaN,NaN
67,gemma2:2b,RTEGLUE,0.7513,0.745253,G2,0.686496,2.316383e-12,8.429312e-12,6.480402,5.875678,NaN,NaN,NaN,ok,ruido,G2
68,gemma2:2b,RTEGLUE,0.7513,0.745253,G3,0.721996,8.573631e-05,2.917039e-03,2.930402,2.325678,NaN,NaN,NaN,ok,ruido,G3


In [633]:
from collections import Counter

In [ ]:
lista_sick=Counter(loss_sig[loss_sig["corpus"]=="SICK"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_snli=Counter(loss_sig[loss_sig["corpus"]=="SNLI"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_rteglue=Counter(loss_sig[loss_sig["corpus"]=="RTEGLUE"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_scitail=Counter(loss_sig[loss_sig["corpus"]=="Scitail"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_diag=Counter(loss_sig[loss_sig["corpus"]=="DIAG"]["missing"].apply(lambda x: tuple(x) ).to_list())

In [635]:
print(lista_sick)
print(lista_snli)
print(lista_rteglue)
print(lista_scitail)
print(lista_diag)

Counter({('G2', 'G4'): 4, ('G1',): 3, ('G1', 'G2'): 3, ('G3', 'G4'): 3, ('G3', 'G2'): 3, ('G3', 'G1'): 3, ('G4',): 2, ('G3',): 2, ('G1', 'G4'): 2})
Counter({('G1', 'G4'): 5, ('G3', 'G1'): 5, ('G1', 'G2'): 5, ('G3', 'G4'): 4, ('G2', 'G4'): 4, ('G1',): 3, ('G2',): 3, ('G3', 'G2'): 3, ('G4',): 2, ('G3',): 1})
Counter({('G1', 'G4'): 3, ('G2', 'G4'): 2, ('G4',): 1, ('G3', 'G4'): 1, ('G3', 'G2'): 1})
Counter({('G3', 'G1'): 2, ('G2', 'G4'): 2, ('G3', 'G2'): 2, ('G4',): 1, ('G1', 'G4'): 1, ('G3',): 1, ('G3', 'G4'): 1, ('G2',): 1, ('G1', 'G2'): 1})
Counter({('G2', 'G4'): 4, ('G3', 'G4'): 2, ('G4',): 1, ('G1', 'G4'): 1})


In [ ]:
lista_g2=Counter(loss_sig[loss_sig["model"]=="gemma2"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_g2_2=Counter(loss_sig[loss_sig["model"]=="gemma2:2b"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_ll31=Counter(loss_sig[loss_sig["model"]=="llama3.1"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_ll32=Counter(loss_sig[loss_sig["model"]=="llama3.2"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_phi3=Counter(loss_sig[loss_sig["model"]=="phi3"]["missing"].apply(lambda x: tuple(x) ).to_list())
lista_phi3_m=Counter(loss_sig[loss_sig["model"]=="phi3:medium"]["missing"].apply(lambda x: tuple(x) ).to_list())

In [637]:
print(lista_g2)
print(lista_g2_2)
print(lista_ll31)
print(lista_ll32)
print(lista_phi3)
print(lista_phi3_m)

Counter({('G3', 'G1'): 2, ('G4',): 1, ('G3',): 1, ('G2',): 1, ('G3', 'G4'): 1, ('G2', 'G4'): 1, ('G1', 'G4'): 1, ('G1', 'G2'): 1})
Counter({('G1', 'G4'): 3, ('G3', 'G4'): 2, ('G2', 'G4'): 2, ('G1',): 2, ('G1', 'G2'): 2, ('G3', 'G1'): 1, ('G4',): 1})
Counter({('G2', 'G4'): 5, ('G3', 'G2'): 4, ('G1', 'G4'): 2, ('G1', 'G2'): 2, ('G1',): 1, ('G3', 'G1'): 1, ('G2',): 1})
Counter({('G2', 'G4'): 3, ('G1', 'G4'): 3, ('G3', 'G1'): 3, ('G3', 'G4'): 3, ('G1',): 2, ('G3', 'G2'): 2, ('G1', 'G2'): 2, ('G4',): 2, ('G3',): 2, ('G2',): 1})
Counter({('G3', 'G4'): 3, ('G4',): 2, ('G1', 'G2'): 2, ('G2',): 1, ('G3', 'G2'): 1, ('G1',): 1, ('G2', 'G4'): 1, ('G3', 'G1'): 1, ('G1', 'G4'): 1})
Counter({('G2', 'G4'): 4, ('G1', 'G4'): 2, ('G3', 'G4'): 2, ('G3', 'G1'): 2, ('G3', 'G2'): 2, ('G4',): 1, ('G3',): 1})


Scitail:  Counter()
RTEGLUE:  Counter({('G1', 'G4'): 1})
SICK:  Counter()
SICK:  Counter()
DIAG:  Counter({('G3', 'G4'): 1, ('G2', 'G4'): 1})


In [683]:
model="phi3"
print('Scitail: ',Counter(loss_sig[(loss_sig["model"]==model) & (loss_sig["corpus"]=="Scitail") & (loss_sig["sensible"]=="sensible") & (loss_sig["robustez"]=="falla")]["missing"].apply(lambda x: tuple(x) ).to_list()))
print('RTEGLUE: ',Counter(loss_sig[(loss_sig["model"]==model) & (loss_sig["corpus"]=="RTEGLUE") & (loss_sig["sensible"]=="sensible") & (loss_sig["robustez"]=="falla")]["missing"].apply(lambda x: tuple(x) ).to_list()))
print('SICK: ',Counter(loss_sig[(loss_sig["model"]==model) & (loss_sig["corpus"]=="SICK") & (loss_sig["sensible"]=="sensible") & (loss_sig["robustez"]=="falla")]["missing"].apply(lambda x: tuple(x) ).to_list()))
print('SICK: ',Counter(loss_sig[(loss_sig["model"]==model) & (loss_sig["corpus"]=="SICK") & (loss_sig["sensible"]=="sensible") & (loss_sig["robustez"]=="falla")]["missing"].apply(lambda x: tuple(x) ).to_list()))
print('DIAG: ',Counter(loss_sig[(loss_sig["model"]==model) & (loss_sig["corpus"]=="DIAG") & (loss_sig["sensible"]=="sensible") & (loss_sig["robustez"]=="falla")]["missing"].apply(lambda x: tuple(x) ).to_list()))
print("-----------------")
print('Scitail: ',Counter(support[(support["model"]==model) & (support["corpus"]=="Scitail") & (support["error"]=="errorGs") & (support["info"]=="información")]["support"].apply(lambda x: tuple(x) ).to_list()))
print('RTEGLUE: ',Counter(support[(support["model"]==model) & (support["corpus"]=="RTEGLUE") & (support["error"]=="errorGs") & (support["info"]=="información")]["support"].apply(lambda x: tuple(x) ).to_list()))
print('SICK: ',Counter(support[(support["model"]==model) & (support["corpus"]=="SICK") & (support["error"]=="errorGs") & (support["info"]=="información")]["support"].apply(lambda x: tuple(x) ).to_list()))
print('SNLI: ',Counter(support[(support["model"]==model) & (support["corpus"]=="SNLI") & (support["error"]=="errorGs") & (support["info"]=="información")]["support"].apply(lambda x: tuple(x) ).to_list()))
print('DIAG: ',Counter(support[(support["model"]==model) & (support["corpus"]=="DIAG") & (support["error"]=="errorGs") & (support["info"]=="información")]["support"].apply(lambda x: tuple(x) ).to_list()))

Scitail:  Counter()
RTEGLUE:  Counter({('G1', 'G4'): 1})
SICK:  Counter()
SICK:  Counter()
DIAG:  Counter({('G3', 'G4'): 1})
-----------------
Scitail:  Counter({('G', '3'): 1})
RTEGLUE:  Counter()
SICK:  Counter()
SNLI:  Counter()
DIAG:  Counter()
